In [ ]:
from forward import solve_forward
from simulation import Simulation
import numpy as np
import mat73
#from net import EEG_CNN

In [ ]:
fwd = solve_forward(downsampled_dipoles=False)
# eeg_topos = mat73.loadmat('../../Downloads/eeg_topos.mat')['eeg_topos']
# eeg_topos = eeg_topos.transpose(2, 0, 1)
# sources = np.load('../../Downloads/sources.npy')
# eeg = np.load('../../Downloads/eeg.npy')
# sim = Simulation(fwd,source_data=sources,eeg_data=eeg)
# sim.simulate(n_samples=100)

In [ ]:
from net import EEGLargeCnn

eegcnn = EEGLargeCnn(directory_y='../../Downloads/sim/source_data', directory_x='../../Downloads/sim/topo_data',
    directory_y_eval='../../Downloads/sim/source_data', directory_x_eval='../../Downloads/sim/topo_data'
)

In [ ]:
eegcnn.build_model()
eegcnn.fit(epochs=5)

In [3]:
import keras_preprocessing_custom.image
directory_x = '/media/thanos/Elements/thanos/sim_data/topos'
directory_y = '/media/thanos/Elements/thanos/sim_data/sources'
loader = keras_preprocessing_custom.image.DataLoader()


train_loader = loader.flow_from_directory(
    directory_y=directory_y,
    directory_x=directory_x,
    batch_size=32,
)       

/media/thanos/Elements/thanos/sim_data/topos/topos_eval


AttributeError: X and Y must have the same amount of samples.